# CO2 equilibrium runs

We'll mimic abrupt 4xCO2 runs from all 66 CMIP6 model calibrations and create Gregory plots.

In [ ]:
import copy

import matplotlib.pyplot as pl
import numpy as np
import pandas as pd

from fair21 import SpeciesID, Category, RunMode, Species, Scenario, ClimateResponse, SpeciesConfig, Config, FAIR
from fair21.forcing.ghg import calculate_ghg_forcing
from fair21.defaults.default_species_config import default_species_config

## 1. start by defining SpeciesID

Today we're running CO2 forcing-driven. Each model is given the same concentration time series, but has a different forcing. We have calibrations of 4xCO2 forcing from the fitting procedure.

At the moment, because of the forcing relationship we also have to define CH4 and N2O if we want to run CO2 concentration-driven.

In [ ]:
species_ids = {}
species_ids['co2'] = SpeciesID('CO2', Category.CO2, RunMode.CONCENTRATION)
species_ids['ch4'] = SpeciesID('CH4', Category.CH4, RunMode.CONCENTRATION)
species_ids['n2o'] = SpeciesID('N2O', Category.N2O, RunMode.CONCENTRATION)

## 2. Define Species

Let's run 1000 years of abrupt-4xCO2. Remember we need one species for each config, and this time there's 66 configs. We will obtain 4xCO2 forcing from each, but we need to calculate how the forcing in each model scales to the FaIR default treatment.

In [ ]:
co2, ch4, n2o = (
    default_species_config['co2'].baseline_concentration,
    default_species_config['ch4'].baseline_concentration,
    default_species_config['n2o'].baseline_concentration
)
print(co2, ch4, n2o)

In [ ]:
species_co2 = Species(species_ids['co2'], concentration=np.ones(1000)*4*co2)
species_ch4 = Species(species_ids['ch4'], concentration=np.ones(1000)*ch4)
species_n2o = Species(species_ids['n2o'], concentration=np.ones(1000)*n2o)

In [ ]:
species_co2

# 3. Define Scenario

This is fairly easy: we have one scenario (abrupt-4xCO2) containing three species (CO2, CH4, N2O).

Even though we only have a single scenario, it needs to be a list

In [ ]:
scenarios = [Scenario('abrupt-4xCO2', [species_co2, species_ch4, species_n2o])]
scenarios

## 4. Create ClimateResponse

In [ ]:
df_4xco2 = pd.read_csv('../data/calibration/4xCO2_cummins.csv')
df_4xco2.set_index(['model', 'run'], inplace=True)
df_4xco2#['F_4xCO2']

In [ ]:
climate_response = {}

for i, (index, data) in enumerate(df_4xco2.iterrows()):
    climate_response[index] = ClimateResponse(
        ocean_heat_capacity = np.array([data['C1'], data['C2'], data['C3']]),
        ocean_heat_transfer = np.array([data['kappa1'], data['kappa2'], data['kappa3']]),
        deep_ocean_efficacy = data['epsilon'],
        stochastic_run = True,
        sigma_eta = data['sigma_eta'],
        sigma_xi = data['sigma_xi'],
        gamma_autocorrelation = data['gamma'],
        seed = i*10101
    )

## 5. Create SpeciesConfig

The only thing that is necessary to do here is to scale the CO2 forcing and turn the tropospheric adjustment to 0 to avoid double counting a scaling factor.

In [ ]:
forcing_4co2_fair = calculate_ghg_forcing(
    np.array([4*co2, ch4, n2o]).reshape((1, 1, 1, 3, 1)),
    np.array([co2, ch4, n2o]).reshape((1, 1, 1, 3, 1)),
    np.array([1, 1, 1]).reshape((1, 1, 1, 3, 1)), 
    None,
    0, 1, 2, []
).squeeze()[0]
forcing_4co2_fair

In [ ]:
species_configs = {}

for index, data in df_4xco2.iterrows():
    species_configs[index] = []
    for species_label in ['co2', 'ch4', 'n2o']:
        # NOTE: really important to use copy here, otherwise we are just creating multiple references to the
        # default dict
        species_configs[index].append(copy.copy(default_species_config[species_label]))
        
        # the defaults also contain the SpeciesID that has a default name and run mode. We want to override this.
        species_configs[index][-1].species_id = species_ids[species_label]
        
    # now set specific CO2 options: CO2 is index 0
    species_configs[index][0].tropospheric_adjustment = 0
    species_configs[index][0].scale = data['F_4xCO2'] / forcing_4co2_fair

In [ ]:
species_configs

## 6. Create Configs

In [ ]:
configs = []
for index, data in df_4xco2.iterrows():
    configs.append(Config(index, climate_response[index], species_configs[index]))

## 7. Run FaIR

In [ ]:
# define time vector
time = np.arange(0.5, 1000)

# initialise
fair = FAIR(scenarios, configs, time)

# run
fair.run()

## 8. Show results

Although we can get convincing internal variability for T and N individually, it appears that the stochastic variability is correlated.

In [ ]:
for i, (index, data) in enumerate(df_4xco2.iterrows()):
    pl.plot(time, fair.temperature[:, 0, i, 0, 0])

In [ ]:
for i, (index, data) in enumerate(df_4xco2.iterrows()):
    pl.plot(time, fair.toa_imbalance[:, 0, i, 0, 0])

In [ ]:
for i, (index, data) in enumerate(df_4xco2.iterrows()):
    pl.plot(time[800:], fair.toa_imbalance[800:, 0, i, 0, 0])
pl.axhline(0, color='k')

In [ ]:
fig, ax = pl.subplots(11, 6, figsize=(16, 30))

for i, (index, data) in enumerate(df_4xco2.iterrows()):
    ax[i//6,i%6].scatter(fair.temperature[:, 0, i, 0, 0], fair.toa_imbalance[:, 0, i, 0, 0])
    ax[i//6,i%6].set_xlim(0,13)
    ax[i//6,i%6].set_ylim(-1, 10)
    ax[i//6,i%6].axhline(0, color='k')
    ax[i//6,i%6].set_title(index, fontsize=6)

In [ ]:
fig, ax = pl.subplots(11, 6, figsize=(16, 30))

for i, (index, data) in enumerate(df_4xco2.iterrows()):
    ax[i//6,i%6].scatter(fair.temperature[:150, 0, i, 0, 0], fair.toa_imbalance[:150, 0, i, 0, 0])
    ax[i//6,i%6].set_xlim(0,13)
    ax[i//6,i%6].set_ylim(-1, 10)
    ax[i//6,i%6].axhline(0, color='k')
    ax[i//6,i%6].set_title(index, fontsize=6)

In [ ]:
for i, (index, data) in enumerate(df_4xco2.iterrows()):
    pl.semilogy(time, fair.emissions_array[:, 0, i, 0, 0])

In [ ]:
for i, (index, data) in enumerate(df_4xco2.iterrows()):
    pl.plot(time[200:], fair.emissions_array[200:, 0, i, 0, 0])